In [0]:
import base64

import databricks.sdk
from databricks.sdk.service.workspace import ImportFormat
import nbformat
import numpy as np 

import pandas as pd
import dspy

In [0]:
class Module(dspy.Module):
  def __init__(self, workspace_client):
    self.workspace_client = workspace_client
    self.module_dict = None

  def get_module_dict(self):
    return self.module_dict
  
  def set_module_dict(self, module_dict):
    self.module_dict = module_dict

  def get_code_from_notebooks(self, notebook_path):
    notebook = self.workspace_client.workspace.export(notebook_path, 
                                      format=ImportFormat.JUPYTER)
    ipynb = base64.decodebytes(notebook.content.encode('ascii')).decode("utf-8")
    notebook = nbformat.reads(ipynb, as_version=4)
    code_dict = {}
    x=0
    for cell in notebook.cells:
        if cell.cell_type == 'code' or cell.cell_type == 'markdown':
          code_dict[x] = cell.source
          x+=1
    context = ''.join(code_dict.values())
    return context
  
  def evaluate_responses(self, context):
    answer_list = []
    for k,module in self.module_dict.items():
      response = module(text=context)
      response_dict = {
        'score': response.score,
        'reasoning': response.reasoning,
        'code_snippet': response.code_snippet
      }
      # print(response_dict)
      answer_list.append(response_dict)
    answer_list_df = pd.DataFrame(answer_list)

    return answer_list_df
  
  def get_error_and_answer_dict(self, context_path, human_answers_list, table_name, section, candidate):
  
    context = self.get_code_from_notebooks(context_path)
    answers = self.evaluate_responses(context)
    answers['candidate'] = candidate
    answers['secton'] = section
    answers['score'] = answers['score'].astype('float64')
    spark.createDataFrame(answers).write.mode("append").saveAsTable(table_name)

    model_asnwers_list = answers['score'].tolist()
    diff = [human_answers_list[i] - model_asnwers_list[i] for i in range(len(human_answers_list))]
    
    abs_error = np.abs(diff).sum()
    inaccurate_predictions = len([x for x in diff if x != 0])

    error_dict = {'abs_error': abs_error, 'inaccurate_predictions': inaccurate_predictions, 'context': diff}

    return {
      'error_dict': error_dict,
      'answers_dict': answers
    }